In [14]:
import sys
import logging
import multiprocessing as mp

import pymc as pm
import pymc.sampling_jax as pmjax
import pandas as pd
import numpy as np
import arviz as az
import pytensor.tensor as pt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,LabelBinarizer
import pickle
import numpyro
# from google.colab import drive

In [15]:
# Set working directory
path = '/Users/wenyuzeng/Downloads/'

In [16]:
xls = pd.ExcelFile(path + 'assay_list.xls')

# to read just one sheet to dataframe:
df = []
for i in range(len(xls.sheet_names)):
    df1 = xls.parse(i)
    df.append(df1)


xls2 = pd.ExcelFile(path + 'assay_list2.xls')

for i in range(len(xls2.sheet_names)):
    df1 = xls2.parse(i)
    df.append(df1)




t = pd.DataFrame({'Cell_Line': ['DT40','DT40', 'DT40'], 'ProtocolName':['tox21-dt40-p1_100', 'tox21-dt40-p1_653', 'tox21-dt40-p1_657']})



assay_info_tox21 = pd.read_csv(path + 'assay_meta_data.csv')
assay_info_tox21 = assay_info_tox21.drop(assay_info_tox21.columns[0], axis = 1)
assay_info_tox21['Cell_Line'] = assay_info_tox21['Cell_Line'].str.replace(r'*', '', regex=True)
assay_info_tox21 = assay_info_tox21[['Cell_Line', 'ProtocolName']]
assay_info_tox21 = assay_info_tox21.drop([49])
assay_info_tox21 = pd.concat([assay_info_tox21, t]).reset_index(drop=True)




assay_info = pd.read_excel(path + 'SampleMeta_Data_update.xlsx')
assay_info = assay_info.drop(assay_info.columns[6:], axis=1).iloc[:13,:].rename(columns = {assay_info.columns[5]: 'Tissue_Type2'})
assay_info = assay_info.rename(columns = {assay_info.columns[0]: 'Cell_Line'})
assay_info['Cell_Line'] = assay_info['Cell_Line'].str.replace(r'*', '', regex=True).str.replace(' ', '')
assay_info['Cell_Type'] = assay_info['Cell_Type'].str.replace(r'*', '', regex=True)
assay_info = assay_info.join(assay_info_tox21.set_index('Cell_Line'), on='Cell_Line')


df_list = []
for i in range(len(df)):
    one_assay = df[i]
    one_assay = one_assay.drop(one_assay.columns[0], axis=1).drop_duplicates()
    one_assay['ProtocolName'] = one_assay.columns[0]
    one_assay = one_assay.rename(columns = {one_assay.columns[0]: 'Outcome'})
    df_list.append(one_assay)
    



df_stack_50 = pd.concat(df_list)
df_stack_50 = df_stack_50.join(assay_info.set_index('ProtocolName'), on='ProtocolName')
# print(df_stack_50.iloc[:,206:])

# ### Train/Test Split & Bayesian model coordination


df_stack_50.Outcome = df_stack_50.Outcome.astype(object)

# Scale numeric chemical descriptors
numeric_columns=list(df_stack_50.select_dtypes(['float64', 'int64']).columns)
categorical_columns=list(df_stack_50.select_dtypes('object').columns)
# print(len(numeric_columns))

pipeline=ColumnTransformer([
    ('num',StandardScaler(),numeric_columns),
    ('cat', 'passthrough', categorical_columns)])

chem_des_scale = pipeline.fit_transform(df_stack_50)
chem_des_scale = pd.DataFrame(chem_des_scale)
# print(chem_des_scale.iloc[:,206:])
col_name = list(df_stack_50.columns)
col_name.pop(0)
col_name.pop(0)
col_name.insert(204, 'Outcome')
col_name.insert(205,'SMILES')
# print('After changing column names')
chem_des_scale.columns = col_name
# print(chem_des_scale.iloc[:,206:])


df_try = chem_des_scale.iloc[:100, :]
factorized_protocols = pd.factorize(chem_des_scale.ProtocolName)

X = chem_des_scale.iloc[:, :204]
# X = chem_des_scale.iloc[:, :20]
y = chem_des_scale['Outcome']
assay_info_try = chem_des_scale.iloc[:,206:]

In [17]:
X

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1.584378,-0.679719,1.583453,-0.31427,-0.67622,2.125535,2.086785,2.127852,2.356477,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
1,-2.006412,1.246915,-2.006851,1.998691,-0.03779,-0.646331,-0.629139,-0.645345,-0.610342,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
2,0.410452,-0.383746,0.409685,0.103053,0.462385,-0.234939,-0.181232,-0.233449,-0.201125,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,14.261846,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
3,0.903448,0.28479,0.902615,-0.481793,0.311839,0.683551,0.649686,0.685182,0.856017,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
4,0.939689,-1.639065,0.938851,-0.597774,0.151686,0.932964,0.939746,0.934421,0.992422,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362395,-1.985526,1.229606,-1.985969,1.9341,-0.630667,-0.309796,-0.353282,-0.31443,-0.508038,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,2.339909,-0.156202
362396,0.887955,0.507896,0.887124,-0.593624,-0.919065,0.555294,0.527465,0.553601,0.549105,-0.055276,...,4.460965,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,0.153651,-0.156202
362397,-1.732884,1.045131,-1.733361,1.245708,0.84016,-0.10922,-0.128097,-0.107954,0.003483,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202
362398,0.717855,0.542259,0.717047,-0.630824,0.887917,-0.337949,-0.303341,-0.339703,-0.405734,-0.055276,...,-0.171002,-0.167359,-0.081049,-0.070021,-0.067972,-0.110237,-0.039508,-0.094127,-0.283601,-0.156202


In [18]:
feature_top_40 = ['fr_C_O', 'TPSA', 'SMR_VSA1', 'NumHAcceptors', 'NumAromaticCarbocycles','NumHeteroatoms',
                  'Chi4n', 'VSA_EState10', 'VSA_EState3', 'Kappa1', 'Chi2v', 'SMR_VSA10', 'SlogP_VSA2',
                  'HallKierAlpha', 'SMR_VSA5', 'VSA_EState7', 'FractionCSP3', 'SlogP_VSA10', 'BCUT2D_LOGPLOW',
                  'VSA_EState2', 'FpDensityMorgan2', 'fr_phenol_noOrthoHbond', 'SlogP_VSA1', 'PEOE_VSA1',
                  'fr_Ar_N', 'BCUT2D_CHGLO', 'fr_ether', 'EState_VSA1', 'MinPartialCharge', 'VSA_EState6',
                  'MaxPartialCharge', 'MinAbsPartialCharge', 'VSA_EState9', 'SMR_VSA3', 'FpDensityMorgan3',
                  'SlogP_VSA12', 'fr_Al_OH_noTert', 'BalabanJ', 'MaxAbsPartialCharge', 'EState_VSA10']
from sklearn import preprocessing, metrics
from imblearn.under_sampling import RandomUnderSampler, NearMiss

In [19]:
label_encoder = preprocessing.LabelEncoder()
# Training
bal_accs_t = []
auc_scores_t = []
recalls_t = []
f1s_t = []
precisions_t = []

# Validation
bal_accs_v = []
auc_scores_v = []
recalls_v = []
f1s_v = []
precisions_v = []

# Testing
bal_accs = []
auc_scores = []
f1_scores = []
recall_scores = []
precision_scores = []

# Others
assays_level_model = []

for z in [0, 1, 2, 3, 4]:
    scaler = StandardScaler()
    y = label_encoder.fit_transform(chem_des_scale['Outcome'])
    X = X[X.columns.intersection(feature_top_40)]
    
    X_assay_info = pd.concat([X, assay_info_try], axis=1)
#     print(X_assay_info)

    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_assay_info, y, 
                                                                    test_size=0.20, shuffle=True, random_state = 42)
    
    X_train, X_valid, y_train, y_valid= train_test_split(X_train_valid, y_train_valid,
                                                         test_size=0.25, shuffle=True, random_state = z)# 5 CV on training

    near_miss = RandomUnderSampler(random_state = z) 
    X_train_resampled, y_train_resampled = near_miss.fit_resample(X_train, y_train)# undersampling

    X_train_resample = X_train_resampled.iloc[:,:40]
    assay_info_resampled = X_train_resampled.iloc[:,40:]

    X_valid_resample = X_valid.iloc[:, :40]
    assay_info_valid = X_valid.iloc[:,40:]
    
    X_test_resample = X_test.iloc[:,:40]
    assay_info_test = X_test.iloc[:,40:]
    
    X_train = scaler.fit_transform(X_train_resample)
    X_valid = scaler.transform(X_valid_resample)
    X_valid = np.array(X_valid)
    X_test = scaler.transform(X_test_resample)
    X_test = np.array(X_test)
    
    print(len(X_train_resample))
    print(len(assay_info_resampled))


    factorized_protocols = pd.factorize(assay_info_resampled.ProtocolName)

    coords_simulated = {
        'obs_id': np.arange(X_train.shape[0]),
        'chem_descrip': np.arange(X_train.shape[1]),
        'protocol':list(factorized_protocols[1]),
        'params':['beta_{0}'.format(j) for j in range(X_train.shape[1])]
    }

    X4_bayes = np.asarray(X_train, dtype="float64")
    Y4_bayes = list(y_train_resampled)
    organisms = pd.factorize(assay_info_resampled.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_resampled.drop_duplicates().Gender)

    with pm.Model(coords=coords_simulated) as assay_level_model:
        x = pm.Data('x', X4_bayes, mutable = True)
        protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
        organism_idx = pm.Data("organism_idx", organisms[0], mutable=True)
        gender_idx = pm.Data("gender_idx", genders[0], mutable=True)
        y = pm.Data('y', Y4_bayes, mutable=True)

        sd_dist = pm.Exponential.dist(1.0)

                # get back standard deviations and rho:
        chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

                #hyperpriors and priors for average betas:
        beta_list = []
        for i in range(X4_bayes.shape[1]):
            gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=3)
            mu_gbeta = gbeta[0] + gbeta[1] * organism_idx + gbeta[2] * gender_idx
            sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
            betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
            beta_list.append(betas)

                #population of varying protocol effects:
        beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

                #Expected value per protocol:
        theta = beta_protocol[protocol_idx,0]* x[:,0]

        for i in range(1,X4_bayes.shape[1]):
            theta += beta_protocol[protocol_idx,i] * x[:,i]

        p = 1.0 / (1.0 + pt.exp(-theta))
        likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)

    assays_level_model.append(assay_level_model)

    with assay_level_model:
        tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)


        # Model Performance
        #Training
    with assay_level_model:
        tr_assay.extend(pm.sample_posterior_predictive(tr_assay))

    train_pred = pd.DataFrame(tr_assay.posterior_predictive['likelihood'].mean(('chain', 'draw')), columns = ['prob'])
    train_pred['round'] = train_pred.prob.round()

    bal_acc_t = metrics.balanced_accuracy_score(list(y_train_resampled), list(train_pred['round']))
    auc_score_t = metrics.roc_auc_score(list(y_train_resampled), list(train_pred['prob']))                               
    recall_t = metrics.recall_score(y_train_resampled, list(train_pred['round']))
    f1_t = metrics.f1_score(y_train_resampled, list(train_pred['round']))
    precision_t = metrics.precision_score(y_train_resampled, list(train_pred['round']))

    bal_accs_t.append(bal_acc_t)
    auc_scores_t.append(auc_score_t)
    recalls_t.append(recall_t)
    f1s_t.append(f1_t)
    precisions_t.append(precision_t)

        #Generate Testing data
    protocol_pred = pd.factorize(assay_info_test.ProtocolName)
    organisms = pd.factorize(assay_info_test.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_test.drop_duplicates().Gender)

    with assay_level_model:
        pm.set_data(
            new_data = {'x': X_test,
                        "protocol_idx": list(protocol_pred[0]),
                        "organism_idx": organisms[0],
                        "gender_idx": genders[0],
                        "y": y_test
                        }
        )
        posterior_predictive = pm.sample_posterior_predictive(
            tr_assay, var_names=["likelihood"],
            predictions = True
        )

        #Testing Performance
    y_pred = pd.DataFrame(posterior_predictive.predictions['likelihood'].mean(('chain', 'draw')),columns=['prob'])
    y_pred['round'] = y_pred.prob.round()

    bal_acc = metrics.balanced_accuracy_score(list(y_test), list(y_pred['round']))
    auc_roc = metrics.roc_auc_score(list(y_test), y_pred['prob'])
    recall = metrics.recall_score(list(y_test), list(y_pred['round']))
    f1 = metrics.f1_score(list(y_test), list(y_pred['round']))
    precision = metrics.precision_score(list(y_test), list(y_pred['round']))

    bal_accs.append(bal_acc)
    auc_scores.append(auc_roc)
    f1_scores.append(f1)
    recall_scores.append(recall)
    precision_scores.append(precision)

        #Generate Validation data
    protocol_valid = pd.factorize(assay_info_valid.ProtocolName)
    organisms = pd.factorize(assay_info_valid.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_valid.drop_duplicates().Gender)

    with assay_level_model:
        pm.set_data(
            new_data = {'x': X_valid,
                        "protocol_idx": list(protocol_valid[0]),
                        "organism_idx": organisms[0],
                        "gender_idx": genders[0],
                        "y": y_valid
                        }
        )
        posterior_valid = pm.sample_posterior_predictive(
            tr_assay, var_names=["likelihood"],
            predictions = True
        )

        #Validation
    y_pred_v = pd.DataFrame(posterior_valid.predictions['likelihood'].mean(('chain', 'draw')),columns=['prob'])
    y_pred_v['round'] = y_pred_v.prob.round()
    
    bal_acc_v = metrics.balanced_accuracy_score(list(y_valid), list(y_pred_v['round']))
    auc_roc_v = metrics.roc_auc_score(list(y_valid), y_pred_v['prob'])
    recall_v = metrics.recall_score(list(y_valid), list(y_pred_v['round']))
    f1_v = metrics.f1_score(list(y_valid), list(y_pred_v['round']))
    precision_v = metrics.precision_score(list(y_valid), list(y_pred_v['round']))

    bal_accs_v.append(bal_acc_v)
    auc_scores_v.append(auc_roc_v)
    recalls_v.append(recall_v)
    f1s_v.append(f1_v)
    precisions_v.append(precision_v)
    
    print(z)


44664
44664
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:32.127949
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

...

Sampling time =  11:46:53.988772
Transforming variables...
Transformation time =  0:00:00.163359


Sampling: [likelihood]
INFO:pymc.sampling.forward:Sampling: [likelihood]


Sampling: [beta_0, beta_1, beta_10, beta_11, beta_12, beta_13, beta_14, beta_15, beta_16, beta_17, beta_18, beta_19, beta_2, beta_20, beta_21, beta_22, beta_23, beta_24, beta_25, beta_26, beta_27, beta_28, beta_29, beta_3, beta_30, beta_31, beta_32, beta_33, beta_34, beta_35, beta_36, beta_37, beta_38, beta_39, beta_4, beta_5, beta_6, beta_7, beta_8, beta_9, beta_protocol, likelihood]
INFO:pymc.sampling.forward:Sampling: [beta_0, beta_1, beta_10, beta_11, beta_12, beta_13, beta_14, beta_15, beta_16, beta_17, beta_18, beta_19, beta_2, beta_20, beta_21, beta_22, beta_23, beta_24, beta_25, beta_26, beta_27, beta_28, beta_29, beta_3, beta_30, beta_31, beta_32, beta_33, beta_34, beta_35, beta_36, beta_37, beta_38, beta_39, beta_4, beta_5, beta_6, beta_7, beta_8, beta_9, beta_protocol, likelihood]


Sampling: [beta_0, beta_1, beta_10, beta_11, beta_12, beta_13, beta_14, beta_15, beta_16, beta_17, beta_18, beta_19, beta_2, beta_20, beta_21, beta_22, beta_23, beta_24, beta_25, beta_26, beta_27, beta_28, beta_29, beta_3, beta_30, beta_31, beta_32, beta_33, beta_34, beta_35, beta_36, beta_37, beta_38, beta_39, beta_4, beta_5, beta_6, beta_7, beta_8, beta_9, beta_protocol, likelihood]
INFO:pymc.sampling.forward:Sampling: [beta_0, beta_1, beta_10, beta_11, beta_12, beta_13, beta_14, beta_15, beta_16, beta_17, beta_18, beta_19, beta_2, beta_20, beta_21, beta_22, beta_23, beta_24, beta_25, beta_26, beta_27, beta_28, beta_29, beta_3, beta_30, beta_31, beta_32, beta_33, beta_34, beta_35, beta_36, beta_37, beta_38, beta_39, beta_4, beta_5, beta_6, beta_7, beta_8, beta_9, beta_protocol, likelihood]


4


In [20]:
print(np.mean(bal_accs))
print(np.std(bal_accs))

print(np.mean(auc_scores))
print(np.std(auc_scores))

print(np.mean(recall_scores))
print(np.std(recall_scores))

print(np.mean(f1_scores))
print(np.std(f1_scores))

print(np.mean(precision_scores))
print(np.std(precision_scores))

0.7375502037861839
0.005352553949063401
0.8137114452431723
0.0027753205381204457
0.7200918794757465
0.013031488595284258
0.3716991724629647
0.003807164395560752
0.25051448190600906
0.002203293081002504


In [22]:
#Training
print(np.mean(bal_accs_t))
print(np.std(bal_accs_t))

print(np.mean(auc_scores_t))
print(np.std(auc_scores_t))

print(np.mean(recalls_t))
print(np.std(recalls_t))

print(np.mean(f1s_t))
print(np.std(f1s_t))

print(np.mean(precisions_t))
print(np.std(precisions_t))

0.7537826441037199
0.0013176049930137231
0.8285773585926034
0.0007859726367384373
0.7419924760735651
0.0022755109886100694
0.7508437877386626
0.0015153648726284762
0.7599118906464952
0.0012296799409434894


In [23]:
#Validation
print(np.mean(bal_accs_v))
print(np.std(bal_accs_v))

print(np.mean(auc_scores_v))
print(np.std(auc_scores_v))

print(np.mean(recalls_v))
print(np.std(recalls_v))

print(np.mean(f1s_v))
print(np.std(f1s_v))

print(np.mean(precisions_v))
print(np.std(precisions_v))

0.731729963118563
0.008785801848799513
0.8059474778012138
0.008533671547330488
0.7167636299600657
0.011853309972747887
0.36644428688660896
0.013987310585020913
0.2463472182279439
0.012563906386261226


In [24]:
label_encoder = preprocessing.LabelEncoder()
# Training
bal_accs_t = []
auc_scores_t = []
recalls_t = []
f1s_t = []
precisions_t = []

# Validation
bal_accs_v = []
auc_scores_v = []
recalls_v = []
f1s_v = []
precisions_v = []

# Testing
bal_accs = []
auc_scores = []
f1_scores = []
recall_scores = []
precision_scores = []

# Others
assays_level_model = []

for z in [0, 1, 2, 3, 4]:
    scaler = StandardScaler()
    y = label_encoder.fit_transform(chem_des_scale['Outcome'])
    X = X[X.columns.intersection(feature_top_40)]
    
    X_assay_info = pd.concat([X, assay_info_try], axis=1)
#     print(X_assay_info)

    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_assay_info, y, 
                                                                    test_size=0.20, shuffle=True, random_state = 42)
    
    X_train, X_valid, y_train, y_valid= train_test_split(X_train_valid, y_train_valid,
                                                         test_size=0.25, shuffle=True, random_state = z)# 5 CV on training

    near_miss = RandomUnderSampler(random_state = z) 
    X_train_resampled, y_train_resampled = near_miss.fit_resample(X_train, y_train)# undersampling

    X_train_resample = X_train_resampled.iloc[:,:40]
    assay_info_resampled = X_train_resampled.iloc[:,40:]

    X_valid_resample = X_valid.iloc[:, :40]
    assay_info_valid = X_valid.iloc[:,40:]
    
    X_test_resample = X_test.iloc[:,:40]
    assay_info_test = X_test.iloc[:,40:]
    
    X_train = scaler.fit_transform(X_train_resample)
    X_valid = scaler.transform(X_valid_resample)
    X_valid = np.array(X_valid)
    X_test = scaler.transform(X_test_resample)
    X_test = np.array(X_test)
    
    print(len(X_train_resample))
    print(len(assay_info_resampled))


    factorized_protocols = pd.factorize(assay_info_resampled.ProtocolName)

    coords_simulated = {
        'obs_id': np.arange(X_train.shape[0]),
        'chem_descrip': np.arange(X_train.shape[1]),
        'protocol':list(factorized_protocols[1]),
        'params':['beta_{0}'.format(j) for j in range(X_train.shape[1])]
    }

    X4_bayes = np.asarray(X_train, dtype="float64")
    Y4_bayes = list(y_train_resampled)
    organisms = pd.factorize(assay_info_resampled.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_resampled.drop_duplicates().Gender)

    with pm.Model(coords=coords_simulated) as assay_level_model:
        x = pm.Data('x', X4_bayes, mutable = True)
        protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
#         organism_idx = pm.Data("organism_idx", organisms[0], mutable=True)
#         gender_idx = pm.Data("gender_idx", genders[0], mutable=True)
        y = pm.Data('y', Y4_bayes, mutable=True)

        sd_dist = pm.Exponential.dist(1.0)

                # get back standard deviations and rho:
        chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

                #hyperpriors and priors for average betas:
        beta_list = []
        for i in range(X4_bayes.shape[1]):
            gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=1)
            mu_gbeta = gbeta[0] 
            sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
            betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
            beta_list.append(betas)

                #population of varying protocol effects:
        beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

                #Expected value per protocol:
        theta = beta_protocol[protocol_idx,0]* x[:,0]

        for i in range(1,X4_bayes.shape[1]):
            theta += beta_protocol[protocol_idx,i] * x[:,i]

        p = 1.0 / (1.0 + pt.exp(-theta))
        likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)

    assays_level_model.append(assay_level_model)

    with assay_level_model:
        tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)


        # Model Performance
        #Training
    with assay_level_model:
        tr_assay.extend(pm.sample_posterior_predictive(tr_assay))

    train_pred = pd.DataFrame(tr_assay.posterior_predictive['likelihood'].mean(('chain', 'draw')), columns = ['prob'])
    train_pred['round'] = train_pred.prob.round()

    bal_acc_t = metrics.balanced_accuracy_score(list(y_train_resampled), list(train_pred['round']))
    auc_score_t = metrics.roc_auc_score(list(y_train_resampled), list(train_pred['prob']))                               
    recall_t = metrics.recall_score(y_train_resampled, list(train_pred['round']))
    f1_t = metrics.f1_score(y_train_resampled, list(train_pred['round']))
    precision_t = metrics.precision_score(y_train_resampled, list(train_pred['round']))

    bal_accs_t.append(bal_acc_t)
    auc_scores_t.append(auc_score_t)
    recalls_t.append(recall_t)
    f1s_t.append(f1_t)
    precisions_t.append(precision_t)

        #Generate Testing data
    protocol_pred = pd.factorize(assay_info_test.ProtocolName)
    organisms = pd.factorize(assay_info_test.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_test.drop_duplicates().Gender)

    with assay_level_model:
        pm.set_data(
            new_data = {'x': X_test,
                        "protocol_idx": list(protocol_pred[0]),
#                         "organism_idx": organisms[0],
#                         "gender_idx": genders[0],
                        "y": y_test
                        }
        )
        posterior_predictive = pm.sample_posterior_predictive(
            tr_assay, var_names=["likelihood"],
            predictions = True
        )

        #Testing Performance
    y_pred = pd.DataFrame(posterior_predictive.predictions['likelihood'].mean(('chain', 'draw')),columns=['prob'])
    y_pred['round'] = y_pred.prob.round()

    bal_acc = metrics.balanced_accuracy_score(list(y_test), list(y_pred['round']))
    auc_roc = metrics.roc_auc_score(list(y_test), y_pred['prob'])
    recall = metrics.recall_score(list(y_test), list(y_pred['round']))
    f1 = metrics.f1_score(list(y_test), list(y_pred['round']))
    precision = metrics.precision_score(list(y_test), list(y_pred['round']))

    bal_accs.append(bal_acc)
    auc_scores.append(auc_roc)
    f1_scores.append(f1)
    recall_scores.append(recall)
    precision_scores.append(precision)

        #Generate Validation data
    protocol_valid = pd.factorize(assay_info_valid.ProtocolName)
    organisms = pd.factorize(assay_info_valid.drop_duplicates().Organism)
    genders = pd.factorize(assay_info_valid.drop_duplicates().Gender)

    with assay_level_model:
        pm.set_data(
            new_data = {'x': X_valid,
                        "protocol_idx": list(protocol_valid[0]),
#                         "organism_idx": organisms[0],
#                         "gender_idx": genders[0],
                        "y": y_valid
                        }
        )
        posterior_valid = pm.sample_posterior_predictive(
            tr_assay, var_names=["likelihood"],
            predictions = True
        )

        #Validation
    y_pred_v = pd.DataFrame(posterior_valid.predictions['likelihood'].mean(('chain', 'draw')),columns=['prob'])
    y_pred_v['round'] = y_pred_v.prob.round()
    
    bal_acc_v = metrics.balanced_accuracy_score(list(y_valid), list(y_pred_v['round']))
    auc_roc_v = metrics.roc_auc_score(list(y_valid), y_pred_v['prob'])
    recall_v = metrics.recall_score(list(y_valid), list(y_pred_v['round']))
    f1_v = metrics.f1_score(list(y_valid), list(y_pred_v['round']))
    precision_v = metrics.precision_score(list(y_valid), list(y_pred_v['round']))

    bal_accs_v.append(bal_acc_v)
    auc_scores_v.append(auc_roc_v)
    recalls_v.append(recall_v)
    f1s_v.append(f1_v)
    precisions_v.append(precision_v)
    
    print(z)


44664
44664
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:17.887208
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

...

Sampling time =  14:10:25.257989
Transforming variables...
Transformation time =  0:00:00.174746


Sampling: [likelihood]
INFO:pymc.sampling.forward:Sampling: [likelihood]


Sampling: [likelihood]
INFO:pymc.sampling.forward:Sampling: [likelihood]


Sampling: [likelihood]
INFO:pymc.sampling.forward:Sampling: [likelihood]


4


In [25]:
# Testing
print(np.mean(bal_accs))
print(np.std(bal_accs))

print(np.mean(auc_scores))
print(np.std(auc_scores))

print(np.mean(recall_scores))
print(np.std(recall_scores))

print(np.mean(f1_scores))
print(np.std(f1_scores))

print(np.mean(precision_scores))
print(np.std(precision_scores))

0.7269222241088633
0.0023111185336662844
0.7997115633912839
0.0013428463696478338
0.7039589244696662
0.003540990315968069
0.36070766859147974
0.0025144185640519332
0.2424776670236856
0.0019304844231933388


In [26]:
#Validation
print(np.mean(bal_accs_v))
print(np.std(bal_accs_v))

print(np.mean(auc_scores_v))
print(np.std(auc_scores_v))

print(np.mean(recalls_v))
print(np.std(recalls_v))

print(np.mean(f1s_v))
print(np.std(f1s_v))

print(np.mean(precisions_v))
print(np.std(precisions_v))

0.7224247062315723
0.004105129729646829
0.7956131847679434
0.0025365624827220633
0.6947663374674512
0.008110535513182724
0.3593855957847327
0.004180521136404754
0.24238703484467625
0.0030310825832394806


In [27]:
#Training
print(np.mean(bal_accs_t))
print(np.std(bal_accs_t))

print(np.mean(auc_scores_t))
print(np.std(auc_scores_t))

print(np.mean(recalls_t))
print(np.std(recalls_t))

print(np.mean(f1s_t))
print(np.std(f1s_t))

print(np.mean(precisions_t))
print(np.std(precisions_t))

0.7534433762374322
0.0007264329919050116
0.8278067601900945
0.0005683307229776988
0.741166543209273
0.0021853191187138424
0.750377602735375
0.0009531269452510313
0.7598265471275785
0.001159525132863076


In [10]:
tissues_4

(array([0, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2,
        1, 2, 2, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 1,
        2, 1, 1, 0, 1, 1]),
 Int64Index([0, 1, 2], dtype='int64'))

### Hierarchical Model with only chemical descriptors

In [ ]:
print('start model setup')
with pm.Model(coords=coords_simulated) as assay_level_model:
    x = pm.Data('x', X4_bayes, mutable = True)
    protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
    y = pm.Data('y', Y4_bayes, mutable=True)

    sd_dist = pm.Exponential.dist(1.0)

        # get back standard deviations and rho:
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

        #hyperpriors and priors for average betas:
    beta_list = []
    for i in range(X4_bayes.shape[1]):
        gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=1)
        mu_gbeta = gbeta[0]
        sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
        betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
        beta_list.append(betas)

        #population of varying protocol effects:
    beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

        #Expected value per protocol:
    theta = beta_protocol[protocol_idx,0]* x[:,0]

    for i in range(1,X4_bayes.shape[1]):
        theta += beta_protocol[protocol_idx,i] * x[:,i]

    p = 1.0 / (1.0 + pt.exp(-theta))
    likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)


print('Done model setup')
    # Run the model
with assay_level_model:
    tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)
print('Finish training model')


start model setup
Done model setup
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:15.224975
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [10]:
pick_insert = open('./trained_model_resampled_nometa.pkl','wb')
pickle.dump(tr_assay, pick_insert)
pick_insert.close()
print('model saved')

model saved


### Hierarchical Model with chemical descriptor and gender

In [9]:
print('model setup for descriptors + gender')
with pm.Model(coords=coords_simulated) as assay_level_model:
    x = pm.Data('x', X4_bayes, mutable = True)
    protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
    gender_idx = pm.Data("gender_idx", gender[0], mutable=True)
    y = pm.Data('y', Y4_bayes, mutable=True)

    sd_dist = pm.Exponential.dist(1.0)

        # get back standard deviations and rho:
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

        #hyperpriors and priors for average betas:
    beta_list = []
    for i in range(X4_bayes.shape[1]):
        gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=2)
        mu_gbeta = gbeta[0] + gbeta[1] * gender_idx
        sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
        betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
        beta_list.append(betas)

        #population of varying protocol effects:
    beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

        #Expected value per protocol:
    theta = beta_protocol[protocol_idx,0]* x[:,0]

    for i in range(1,X4_bayes.shape[1]):
        theta += beta_protocol[protocol_idx,i] * x[:,i]

    p = 1.0 / (1.0 + pt.exp(-theta))
    likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)


print('Done model setup')
    # Run the model
with assay_level_model:
    tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)
print('Finish training model')


model setup for descriptors + gender
Done model setup
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:16.989849
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Sampling time =  13:08:30.026738
Transforming variables...
Transformation time =  0:00:00.339182
Finish training model


In [10]:
pick_insert = open('./trained_model_resampled_gender.pkl','wb')
pickle.dump(tr_assay, pick_insert)
pick_insert.close()
print('model saved')

model saved


In [9]:
print('model setup for descriptors + organism')
with pm.Model(coords=coords_simulated) as assay_level_model:
    x = pm.Data('x', X4_bayes, mutable = True)
    protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
    organism_idx = pm.Data("organism_idx", organisms[0], mutable=True)
    y = pm.Data('y', Y4_bayes, mutable=True)

    sd_dist = pm.Exponential.dist(1.0)

        # get back standard deviations and rho:
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

        #hyperpriors and priors for average betas:
    beta_list = []
    for i in range(X4_bayes.shape[1]):
        gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=2)
        mu_gbeta = gbeta[0] + gbeta[1] * organism_idx
        sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
        betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
        beta_list.append(betas)

        #population of varying protocol effects:
    beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

        #Expected value per protocol:
    theta = beta_protocol[protocol_idx,0]* x[:,0]

    for i in range(1,X4_bayes.shape[1]):
        theta += beta_protocol[protocol_idx,i] * x[:,i]

    p = 1.0 / (1.0 + pt.exp(-theta))
    likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)


print('Done model setup')
    # Run the model
with assay_level_model:
    tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)
print('Finish training model')


model setup for descriptors + organism
Done model setup
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:17.324278
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Sampling time =  13:07:31.551875
Transforming variables...
Transformation time =  0:00:00.503054
Finish training model


In [10]:
pick_insert = open('./trained_model_resampled_organism.pkl','wb')
pickle.dump(tr_assay, pick_insert)
pick_insert.close()
print('model saved')

model saved


In [9]:
print('model setup for descriptors + gender_organism')
with pm.Model(coords=coords_simulated) as assay_level_model:
    x = pm.Data('x', X4_bayes, mutable = True)
    protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
    gender_idx = pm.Data("gender_idx", gender[0], mutable=True)
    organism_idx = pm.Data("organism_idx", organisms[0], mutable=True)
    y = pm.Data('y', Y4_bayes, mutable=True)

    sd_dist = pm.Exponential.dist(1.0)

        # get back standard deviations and rho:
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

        #hyperpriors and priors for average betas:
    beta_list = []
    for i in range(X4_bayes.shape[1]):
        gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=3)
        mu_gbeta = gbeta[0] + gbeta[1] * gender_idx + gbeta[2] * organism_idx
        sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
        betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
        beta_list.append(betas)

        #population of varying protocol effects:
    beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

        #Expected value per protocol:
    theta = beta_protocol[protocol_idx,0]* x[:,0]

    for i in range(1,X4_bayes.shape[1]):
        theta += beta_protocol[protocol_idx,i] * x[:,i]

    p = 1.0 / (1.0 + pt.exp(-theta))
    likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)


print('Done model setup')
    # Run the model
with assay_level_model:
    tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)
print('Finish training model')


model setup for descriptors + gender_organism
Done model setup
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:29.084397
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Sampling time =  23:59:29.471322
Transforming variables...
Transformation time =  0:00:00.492497
Finish training model


In [10]:
pick_insert = open('./trained_model_resampled_gender_organism.pkl','wb')
pickle.dump(tr_assay, pick_insert)
pick_insert.close()
print('model saved')

model saved


### model with tumor type and tissue type

In [34]:
print('model setup for descriptors + tumor_tissue')
with pm.Model(coords=coords_simulated) as assay_level_model:
    x = pm.Data('x', X4_bayes, mutable = True)
    protocol_idx = pm.Data("protocol_idx", list(factorized_protocols[0]), mutable=True)
    tumor_idx = pm.Data("tumor_idx", tissues_4[0], mutable=True)
#     tissue_idx = pm.Data("tissue_idx", tissues_2[0], mutable=True)
    y = pm.Data('y', Y4_bayes, mutable=True)

    sd_dist = pm.Exponential.dist(1.0)

        # get back standard deviations and rho:
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=X4_bayes.shape[1], eta=2.0, sd_dist=sd_dist, compute_corr=True)

        #hyperpriors and priors for average betas:
    beta_list = []
    for i in range(X4_bayes.shape[1]):
        gbeta = pm.Normal("g_beta_{0}".format(i), mu=0.0, sigma=10.0, shape=2)
        mu_gbeta = gbeta[0] + gbeta[1] * tumor_idx 
#         + gbeta[2] * tissue_idx
        sigma_beta = pm.Exponential('sigma_beta_{0}'.format(i), 1.0)
        betas = pm.Normal('beta_{0}'.format(i), mu=mu_gbeta,sigma=sigma_beta,dims="protocol")
        beta_list.append(betas)

        #population of varying protocol effects:
    beta_protocol = pm.MvNormal("beta_protocol", mu=pt.stack(beta_list, axis=1), chol=chol,dims=('protocol', 'params'))

        #Expected value per protocol:
    theta = beta_protocol[protocol_idx,0]* x[:,0]

    for i in range(1,X4_bayes.shape[1]):
        theta += beta_protocol[protocol_idx,i] * x[:,i]

    p = 1.0 / (1.0 + pt.exp(-theta))
    likelihood = pm.Bernoulli('likelihood', p, observed=y, shape = p.shape)


print('Done model setup')
    # Run the model
with assay_level_model:
    tr_assay = pmjax.sample_numpyro_nuts(draws=1000, tune=1000, chains=4)
print('Finish training model')


model setup for descriptors + tumor_tissue
Done model setup
Compiling...


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Compilation time =  0:00:16.977098
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: boolean_indexing_set_or_inc
ERROR:pytensor.graph.rewriting.basic:Rewrite failure due to: boolean_indexing_set_or_inc
ERROR (pytensor.graph.rewriting.basic): node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR:pytensor.graph.rewriting.basic:node: AdvancedSetSubtensor(Alloc.0, AdvancedIncSubtensor1{no_inplace,set}.0, [ 0  1  1 ... 39 39 39], [ 0  0  1 ... 37 38 39])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR:pytensor.graph.rewriting.basic:TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1914, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Users/wenyuzeng/anaconda3/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1074, in transform
    re

Sampling time =  13:03:16.733940
Transforming variables...
Transformation time =  0:00:00.164993
Finish training model


In [35]:
pick_insert = open('./trained_model_resampled_tumor.pkl','wb')
pickle.dump(tr_assay, pick_insert)
pick_insert.close()
print('model saved')

model saved
